In [7]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification
import re
import langid
from tqdm import tqdm




In [3]:
#load data
data = pd.read_csv(r'C:\Users\migui\OneDrive - UFV\4ºBA\TFG\SetDatos\dota2_chat_messages.csv', encoding='utf-8')

In [4]:
print(data.head())  # Muestra las primeras filas
print(data.info())  # Información de columnas y tipos de datos
print(data.describe())  # Estadísticas de las columnas numéricas


   match        time  slot            text
0      0  1005.12122     9       ладно гг 
1      0  1005.85442     9             изи
2      0  1008.65372     9              од
3      0  1010.51992     9          ебаный
4      0  1013.91912     9  мусор на войде
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21659448 entries, 0 to 21659447
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   match   int64  
 1   time    float64
 2   slot    int64  
 3   text    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 661.0+ MB
None
              match          time          slot
count  2.165945e+07  2.165945e+07  2.165945e+07
mean   4.996186e+05  1.457327e+03  4.504226e+00
std    2.888392e+05  9.540682e+02  2.875228e+00
min    0.000000e+00 -1.068088e+03  0.000000e+00
25%    2.493490e+05  7.288557e+02  2.000000e+00
50%    4.994910e+05  1.501327e+03  5.000000e+00
75%    7.498670e+05  2.118162e+03  7.000000e+00
max    9.999990e+05  8.443546e+03  9.000000e+00


In [5]:
# Contar valores nulos por columna
valores_nulos = data.isnull().sum()
print(valores_nulos)

# Contar el total de valores nulos
total_nulos = data.isnull().sum().sum()
print("Total de valores nulos:", total_nulos)

match      0
time       0
slot       0
text     243
dtype: int64
Total de valores nulos: 243


In [6]:
# eliminar valores vacios
data['text'] = data['text'].fillna('')

print('Mean length of text', data['text'].apply(lambda x: len(x)).mean())
print(data.head(15))

Mean length of text 10.344593361751416
    match        time  slot                                text
0       0  1005.12122     9                           ладно гг 
1       0  1005.85442     9                                 изи
2       0  1008.65372     9                                  од
3       0  1010.51992     9                              ебаный
4       0  1013.91912     9                      мусор на войде
5       0  1800.31402     9                              мусор 
6       0  1801.71882     9                            на войде
7       0  1802.98982     9                              репорт
8       0  1808.40822     9                                100%
9       1  -131.14018     0              twitch.tv/rage_channel
10      1  -121.60481     0  https://www.twitch.tv/rage_channel
11      1   244.47367     7                      2 даша подряд 
12      1   249.93900     7                               баша 
13      1   255.00443     4                           где даша?
1

In [ ]:
#Comprobar GPU para usar 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"using device: {device}")

devNum = torch.cuda.current_device()

print(f"device number: {devNum}")

devName = torch.cuda.get_device_name(devNum)

print(f"device name: {devName}")

using device: cuda
device number: 0
device name: NVIDIA GeForce RTX 3060 Ti


In [ ]:

# Eliminar valores nulos y vacíos
data = data.dropna(subset=['text'])  # Elimina filas donde text es NaN
data = data[data['text'].str.strip() != '']  # Elimina filas donde text está vacío

# Función para limpiar el texto
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Elimina URLs
    text = re.sub(r"\d+", "", text)  # Elimina números
    text = re.sub(r"\s*-\s*", " ", text).strip()
    text = re.sub(r"[^\w\s]", "", text)  # Elimina puntuación
    text = re.sub(r"\s+", " ", text).strip()  # Elimina espacios extra
    return text

# Aplicar limpieza
data['cleaned_text'] = data['text'].astype(str).apply(clean_text)

# palabras significativas
significant_words = {
    "gg", "wp", "report", "noob", "help", "team", "mid", "bot", "top", "jungle", "roshan",
    "barracks", "tower", "push", "gank", "ward", "rune", "creeps", "jungler", "carry",
    "support", "tank", "healer", "stun", "silence", "slow", "root", "hex", "dmg", "damage",
    "hp", "mana", "gold", "cs", "lasthit", "denied", "kill", "death", "assist", "kda",
    "lvl", "level", "exp", "experience", "min", "minute", "sec", "second", "hour", "day",
    "week", "month", "year", "win", "lose", "victory", "defeat", "draw", "ban", "pick",
    "draft", "strategy", "meta", "patch", "nerf", "buff", "balance", "op", "broken",
    "trash", "good", "bad", "best", "worst", "fun", "boring", "exciting", "tilt", "rage",
    "toxic", "flame", "feed", "afk", "mute", "kick", "surrender", "ff", "end", "game",
    "match", "round", "lobby", "queue", "rank", "mmr", "elo", "tier", "division",
    "leaderboard", "ranked", "unranked", "casual", "competitive", "esports",
    "tournament", "event", "prize", "money", "stream", "streamer", "youtuber", "pro",
    "newbie", "beginner", "veteran", "tryhard", "serious", "chill", "relax"
}

# filtro mensajes cortos
def filter_messages(message):
    if len(message) <= 3:  # Elimina mensajes con 3 o menos letras
        return False
    if len(message.split()) == 1:  # Si es una palabra de la lista
        return message.lower() in significant_words
    return True  # Mantiene todos los demás mensajes

# aplicar filtro
filtered_data = data[data['cleaned_text'].apply(filter_messages)]

# save
filtered_data.to_csv('filtered_dota2_chats.csv', index=False)

#visualizado
print("Datos después del filtrado:")
print(filtered_data[['text', 'cleaned_text']].sample(15))

Datos después del filtrado:
                                  text                 cleaned_text
16852296            ya y quien jodio ?             ya y quien jodio
8884166                 report jug pls               report jug pls
7178367              снесите нас нахуй            снесите нас нахуй
2957208   CM< Upper left corner of map  CM Upper left corner of map
13536399                  пизди пизди                   пизди пизди
2757151              menya zovut imba              menya zovut imba
19895322        Why did you jump on me       Why did you jump on me
2400567                      que paso                      que paso
17309639                      тим плей                     тим плей
9775230           так что закрой ебало         так что закрой ебало
1061170                 ZONIGN A TECHI               ZONIGN A TECHI
17224058               это новая мета?               это новая мета
14016221                       pick es                      pick es
12196054            

In [ ]:
#funcion para traducir
def translator(src: str, dest: str):
    src = src.lower()
    dest = dest.lower()
    tokenizer = AutoTokenizer\
                    .from_pretrained(f"Helsinki-NLP/opus-mt-{src}-{dest}")
    model = AutoModelForSeq2SeqLM\
                    .from_pretrained(f"Helsinki-NLP/opus-mt-{src}-{dest}")
    return pipeline(task='translation',
                    model=model, tokenizer=tokenizer, device=0) #Device 0 es la GPU



In [ ]:
#prueba de traduccion 50 mensajes
df_sample = filtered_data.sample(50)
translate = translator('ru', 'en')
with torch.no_grad():
    translated_text = translate([t[:100] for t in list(df_sample['cleaned_text'])])
df_sample['text_en'] = [t['translation_text'] for t in translated_text]
df_sample.head(10)


,match,time,slot,text,cleaned_text,text_en
15682629,723913,1636.37956,5,пушить надо,пушить надо,You've got to let it go.
10922896,503757,851.65875,7,gg wp,gg wp,gg wp
8373073,385640,1222.06827,5,ty for this 3 free level,ty for this free level,ty for this free lion
658933,30352,1690.48723,3,fucking joker,fucking joker,fucking joker
2236094,102805,1219.03570,8,че суки,че суки,You're a bitch.
20783191,959501,257.13723,9,ona retard,ona retard,ona retard
4097892,188394,2317.76195,2,"so why aren't you out tonight, hitting on weed...",so why arent you out tonight hitting on weed a...,So who's you out tonight doing it on weed and ...
6002457,276549,3061.16130,3,yeah ez gane,yeah ez gane,yeh ez gane
2786998,128050,1546.52234,4,why unpause guys ?,why unpause guys,Whoy unpause guys
3325732,152984,-11.19726,1,через арроу,через арроу,Through Arrow


In [ ]:


# Función para inicializar el traductor
def translator(src: str, dest: str):
    src = src.lower()
    dest = dest.lower()
    model_name = f"Helsinki-NLP/opus-mt-{src}-{dest}"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    return pipeline(task='translation', model=model, tokenizer=tokenizer, device=0)  # GPU activada

# Inicializar traductor ruso → inglés
translate = translator('ru', 'en')

# Seleccionar una muestra aleatoria de 100.000 registros
df_sample = filtered_data.sample(n=100000, random_state=42)

# Parámetros para procesamiento por lotes
batch_size = 5000
total_rows = len(df_sample)
num_batches = (total_rows // batch_size) + 1  # Número total de lotes

# Lista para almacenar los resultados traducidos
translated_batches = []

# Procesar por lotes
for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, total_rows)  # Evitar salir del índice
    df_batch = df_sample.iloc[start:end].copy()  # Extraer batch

    # Traducción con torch.no_grad() para optimización
    with torch.no_grad():
        translated_text = translate([t[:100] for t in list(df_batch['cleaned_text'])])

    # Agregar traducción a la columna
    df_batch['text_en'] = [t['translation_text'] for t in translated_text]

    # Guardar batch traducido
    translated_batches.append(df_batch)

    print(f"Batch {i+1}/{num_batches} traducido.")

# Combinar todos los lotes en un solo DataFrame
df_translated = pd.concat(translated_batches, ignore_index=True)

# Guardar en archivo CSV
df_translated.to_csv("chats_semifinal.csv", index=False)

# Mostrar los primeros registros traducidos
df_translated.head(10)


Batch 1/21 traducido.
Batch 2/21 traducido.
Batch 3/21 traducido.
Batch 4/21 traducido.
Batch 5/21 traducido.
Batch 6/21 traducido.
Batch 7/21 traducido.
Batch 8/21 traducido.
Batch 9/21 traducido.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Batch 10/21 traducido.
Batch 11/21 traducido.
Batch 12/21 traducido.
Batch 13/21 traducido.
Batch 14/21 traducido.
Batch 15/21 traducido.
Batch 16/21 traducido.
Batch 17/21 traducido.
Batch 18/21 traducido.
Batch 19/21 traducido.
Batch 20/21 traducido.
Batch 21/21 traducido.


,match,time,slot,text,cleaned_text,text_en
0,954334,-26.06113,7,fist item,fist item,Fist item
1,756982,85.44562,9,new meta/),new meta,new meta
2,925441,2173.20525,5,найс у нас керри гг,найс у нас керри гг,we've got kerry gg.
3,499506,-31.82556,9,wd moi dryg,wd moi dryg,wd moi dryg
4,946663,1507.19866,6,блять очкошная,блять очкошная,"Fuck, it's a point."
5,345625,2799.93426,4,nice nice,nice nice,nice nice
6,244781,1298.58290,0,Dont be so nervous lion,Dont be so nervous lion,Dont be so nervous lion
7,848211,52.92040,9,"ну так дела не делаются, пацаны",ну так дела не делаются пацаны,"Well, that's not how you guys do things."
8,32307,1167.28164,2,ya idk,ya idk,ya idk
9,18075,1571.88290,8,мы максимум дока убьём,мы максимум дока убьём,"We're gonna kill the doc, tops."


In [4]:
# Cargar el dataset
data_unknown = pd.read_csv('chats_semifinal.csv')

In [6]:
data_unknown.sample(10)


,match,time,slot,text,cleaned_text,text_en
75545,211253,2576.60745,9,levem logo pq se nao perde,levem logo pq se nao perde,levem log pq se nao perde
45297,155262,651.30765,0,se siente bien no tener support :D,se siente bien no tener support D,se siente bien no tener support D
44129,479008,2564.57160,1,изи мид,изи мид,med and mead
61013,613978,1947.46610,5,todos report despus,todos report despus,todos report despus
25545,761068,760.84754,9,Мне похуй,Мне похуй,I don't give a fuck.
29432,189608,864.95546,0,шлюха бегает стилит,шлюха бегает стилит,Whore running stylish.
17846,565803,2326.08705,4,ez team,ez team,ez team
15904,496627,2686.28015,3,gg end,gg end,gg end
82378,368106,1154.81800,3,i`m comming broo gooo,im comming broo gooo,Im comming broo gooo
566,215874,-92.67251,0,wish i knew,wish i knew,wish i knew


In [ ]:

# Función para detectar idioma con langid
def is_english_langid(text):
    lang, _ = langid.classify(text)  # Detecta idioma
    return lang == 'en'  # Retorna True si es inglés

# Aplicar detección con barra de progreso
tqdm.pandas(desc="Detectando idioma")  
data_unknown['is_english'] = data_unknown['text_en'].astype(str).progress_apply(is_english_langid)

print(" Primeros 30 registros con detección de idioma:")
print(data_unknown[['text_en', 'is_english']].head(30))



Detectando idioma: 100%|██████████| 100000/100000 [02:20<00:00, 712.64it/s]

🔍 Primeros 30 registros con detección de idioma:
                                      text_en  is_english
0                                   Fist item        True
1                                    new meta        True
2                         we've got kerry gg.        True
3                                 wd moi dryg       False
4                         Fuck, it's a point.        True
5                                   nice nice       False
6                     Dont be so nervous lion        True
7    Well, that's not how you guys do things.        True
8                                      ya idk       False
9             We're gonna kill the doc, tops.        True
10                            You're special.        True
11                                 Sorry mate        True
12                    come on lets final this        True
13                          Kim know Him lose        True
14                        Venge REPORT PLEASE        True
15                     

In [9]:
# Mostrar el conteo de textos en inglés vs. no inglés
print(data_unknown['is_english'].value_counts())
print(data_unknown[['text_en', 'is_english']].sample(30))

is_english
True     73577
False    26423
Name: count, dtype: int64
                                             text_en  is_english
28101                      Weaver, you won the game.        True
27411  So uh yea how out we all go back to our lones        True
7209                                          Too ez       False
38459                             or are you fucking        True
93263                                          gg wp        True
42735                           i take fake blowjobs        True
56144                                   Nice ez game       False
28576               I'm gonna go out alone with you.        True
44094                            Don't take it long.        True
27390                           You're fucking down.       False
8702                       But the root of the role.        True
53804            Motherfuckers to the fucking Wards.        True
81831                                  FOR THE RICKS        True
42574                  

In [ ]:
# Filtrar solo los mensajes que están en inglés
filtered_data = data_unknown[data_unknown['is_english']]

# Guardar el nuevo dataset con solo los mensajes en inglés
filtered_data.to_csv('only_english_dota2_chats.csv', index=False)

# 📌 Mensajes eliminados y estadísticas
print("✅ Se han eliminado los mensajes que no están en inglés.")
print(f"Total de mensajes en inglés: {len(filtered_data)}")
print(f"Total de mensajes eliminados: {len(data_unknown) - len(filtered_data)}")

# Mostrar los primeros 10 mensajes filtrados
filtered_data.head(10)
filtered_data.sample(10)

✅ Se han eliminado los mensajes que no están en inglés.
📊 Total de mensajes en inglés: 73577
📉 Total de mensajes eliminados: 26423


,match,time,slot,text,cleaned_text,text_en,is_english
65601,752479,49.68786,9,sentry allreayd?,sentry allreayd,sentry allreid,True
52230,81972,2194.77970,5,IDI NAXUY,IDI NAXUY,IDI NAXUY,True
99702,628702,3179.21763,9,ещё бустить будешь,ещё бустить будешь,You're gonna wake up.,True
19930,447804,1880.46227,8,бом бом,бом бом,"Boom, boom, boom, boom, boom, boom, boom, boom...",True
44298,865448,1005.49424,8,"уебки либо пушите,либо минус ебало",уебки либо пушителибо минус ебало,Fucks or cannons or fucks.,True
34586,193505,3010.18948,0,все на базе,все на базе,It's all at base.,True
97008,363500,2371.02970,3,SACA RADAICNE,SACA RADAICNE,SACA RADAICNE,True
71860,592451,2329.95175,4,лол хахахах,лол хахахах,I'll take care of it.,True
16349,231495,-31.85889,5,You guys are EXCELLENT,You guys are EXCELLENT,You Guys are EXCELLENT,True
86792,141148,312.35708,8,придите на линию,придите на линию,Come to the line.,True
